In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Winter '22/IR/Project/"

 aspectWordMat	      final_df.csv	  rp166f-wang.pdf
 aspectWordMat_json   Full_HotelRec.zip   UniqueHotels.pkl
 FetchedRecords.txt   ProcessedText.txt  'Wang_LARA DFo7R.pdf'


In [ ]:
from itertools import islice

PATH = "/content/drive/My Drive/Winter '22/IR/Project/ProcessedText.txt"

N = 50000
with open(PATH,"r") as myfile:
    record_list = list(islice(myfile, N))
print(len(record_list))

50000


In [ ]:
import json
from collections import defaultdict

rev=""
all_rev=[]
all_rev_txt=[]
all_rev_rating=[]
all_rev_id=[]

for r in range(len(record_list)):
  json_data=json.loads(record_list[r])
  rev+=json_data['text']
  all_rev.append(json_data)
  all_rev_txt.append(json_data['text'])
  all_rev_rating.append(json_data['property_dict'])

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

sentences=[]
review_index=[]

ind=0
for review in all_rev_txt:
  temp1=review.split('.')
  temp2=[]
  #print(temp1)
  for s in temp1:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    temp2.append(word_list)

  temp3=[]
  for s in temp2:
    if(len(s)!=0):
      temp3.append(' '.join(s))
      review_index.append(ind)

  sentences+=temp3
  ind+=1

sentences

['decided family holiday destination saw ranking family hotel review',
 'kept promise',
 'plus staff extremely kind willing help time great attention confort kid private playground front hotel pinery providing shade day long great breakfast beach umbrella deck chair assigned guest beach right front hotel great kid menu also weaning lot activity kid parent well wish free baby club year old swimming pool childrens corner big work fine feel like home two gazebo beach used miniclubs massage available day long spa beach towel provided minus perfection impossible restaurant tends bit slow beach gazebo bigger kid bit small midday sun couple note dont mind first floor avoid elevator peak time queing booking bit messy better double check make sure everythings fine bed unconfortable try get changed known would done overall great time would deserve star service perspective',
 'great customer service good restaurant service made experience wonderful family',
 'giulianuova pretty simple wow town cl

In [ ]:
#sentences=nltk.sent_tokenize(rev)
sentence_list=[]
for s in sentences:
  sentence=s.lower()
  words=word_tokenize(s)
  en_stopwords = stopwords.words('english')
  processed_words=[]
  for token in words:
        if token not in en_stopwords:
            processed_words.append(token)

  tokenizer= RegexpTokenizer(r"\w+")
  word_list=tokenizer.tokenize(' '.join(processed_words))
  word_list=[w.lower() for w in word_list if len(w)!=1]
  sentence_list.append(word_list)

sentences=[]
for s in sentence_list:
  if(len(s)!=0):
    sentences.append(' '.join(s))

sentence_wordfreq=[]

for i in sentence_list:
  sentence_wordfreq.append(FreqDist(i))

sentence_wordfreq

[FreqDist({'decided': 1,
           'destination': 1,
           'family': 2,
           'holiday': 1,
           'hotel': 1,
           'ranking': 1,
           'review': 1,
           'saw': 1}),
 FreqDist({'kept': 1, 'promise': 1}),
 FreqDist({'activity': 1,
           'also': 1,
           'assigned': 1,
           'attention': 1,
           'available': 1,
           'avoid': 1,
           'baby': 1,
           'beach': 5,
           'bed': 1,
           'better': 1,
           'big': 1,
           'bigger': 1,
           'bit': 3,
           'booking': 1,
           'breakfast': 1,
           'chair': 1,
           'changed': 1,
           'check': 1,
           'childrens': 1,
           'club': 1,
           'confort': 1,
           'corner': 1,
           'couple': 1,
           'day': 2,
           'deck': 1,
           'deserve': 1,
           'done': 1,
           'dont': 1,
           'double': 1,
           'elevator': 1,
           'everythings': 1,
           'extremely

In [ ]:
from typing import DefaultDict
aspect_keyword=DefaultDict()

aspect_keyword["value"]=["value","price"]
aspect_keyword["room"]=["room","space"]
aspect_keyword["location"]=["location","locate"]
aspect_keyword["cleanliness"]=["clean","dirty"]
aspect_keyword["service"]=["service","manager"]

aspect_keyword

defaultdict(None,
            {'cleanliness': ['clean', 'dirty'],
             'location': ['location', 'locate'],
             'room': ['room', 'space'],
             'service': ['service', 'manager'],
             'value': ['value', 'price']})

functions needed for aspectsegmentation

In [ ]:
all_words=[]

for s in sentences:
  for w in s.split():
    if w not in all_words:
      all_words.append(w)


In [ ]:
from collections import defaultdict

def assign_aspect(wordfreq):
  sentence_aspect=[]
  count=DefaultDict(int)
  for word in wordfreq.keys():
    for aspect, keywords in aspect_keyword.items():
      if word in keywords:
        count[aspect]+=1
  if count : 
    max = max(count.values())
    for aspect, c in count.items():
      if c==max:
        sentence_aspect.append(aspect)

  return sentence_aspect


In [ ]:
import copy

sentences_dup=copy.deepcopy(sentences)

Aspect Segmentation

In [ ]:
from collections import defaultdict
import heapq

aspectWordMat = defaultdict(lambda: defaultdict(int))
aspectCount= DefaultDict(int)
wordCount=DefaultDict(int)

i=1000
changed=True
while changed:
  i-=1
  sentence_aspect=[]
  for word_freq in sentence_wordfreq:
    sentence_aspect.append(assign_aspect(word_freq))

  aspect_to_sentence=DefaultDict(list)
  aspect_to_sentence.clear()
  for i in range(len(sentences)):
    if len(sentence_aspect[i])==1:
      aspect_to_sentence[sentence_aspect[i][0]].append(sentences[i])

  #print(aspect_to_sentence)

  for aspect, sentences in aspect_to_sentence.items():
    for i in range(len(sentences)):
      for word,freq in sentence_wordfreq[i].items():
        aspectWordMat[aspect][word]+=freq
        aspectCount[aspect]+=freq
        wordCount[word]+=freq


  top_p_words = {}

  for aspect in aspect_keyword.keys():
    top_p_words[aspect] = []
  for word in all_words:
      maxChi = 0.0 
      maxAspect = "" 
      for aspect in aspect_keyword.keys():
        sum=0
        for val in aspectCount.values():
          sum+=val
        C=sum
        C1 = aspectWordMat[aspect][word]
        C2 = wordCount[word]-C1
        C3 = aspectCount[aspect]-C1 
        C4 = C-C1 
        d = (C1+C3)*(C2+C4)*(C1+C2)*(C3+C4)
        if d!=0:
          chisq = (C*(C1*C4 - C2*C3)*(C1*C4 - C2*C3))/d
        else:
          chisq = 0.0

        aspectWordMat[aspect][word] = chisq
        if aspectWordMat[aspect][word] > maxChi:
          maxChi = aspectWordMat[aspect][word]
          maxAspect = aspect
      if maxAspect!="":
        top_p_words[maxAspect].append((maxChi, word))

  changed=False
  for aspect in aspect_keyword.keys():
    word_tup=top_p_words[aspect]
    max_words=[]
    max_val=0.0
    for i in range(5):
      for tup in word_tup:
        if tup[0]>max_val and (tup[1] not in max_words) :
          max_val=tup[0]
          max_word=tup[1]
      max_words.append(max_word)
      #word_tup.remove((max_val,max_word))

    for word in max_words:
      if word not in aspect_keyword[aspect]:
        changed=True
        aspect_keyword[aspect].append(word)


In [ ]:
print(aspect_keyword)

defaultdict(None, {'value': ['value', 'price', 'tiamo', 'resort', 'fishing', 'basel', 'beach', 'eiffel', 'tower', 'zahara', 'charles', 'papagayo', 'mkts', 'michels', 'michel', 'rivedoux', 'flotte', 'slam', 'nonslip', 'oyster', 'ubahn', 'renato', 'toll', 'sol', 'rum', 'berlin', 'ile', 'pienza', 'parking', 'kid', 'train', 'mosaico', 'child'], 'room': ['room', 'space', 'hotel', 'stay', 'clean', 'staff', 'good', 'movie', 'breakfast', 'one', 'kayumanis', 'williams', 'nusa', 'dua', 'sofitel', 'barcelona', 'scott', 'nonsmoking', 'sunrise', 'winery', 'motor', 'figured', 'motel', 'takeaway', 'eve', 'tlc', 'nasty', 'disaster', 'great', 'time', 'day', 'night', 'restaurant'], 'location': ['location', 'locate', 'cabin', 'ankara', 'museum', 'ben', 'haveli', 'room', 'staff', 'stay', 'great', 'clean', 'kotra', 'udaipur', 'highland', 'ian', 'midge', 'crianlarich', 'citadel', 'anatolian', 'han', 'civilisation', 'chatted', 'divan', 'butterfly', 'atn', 'transfert', 'tc', 'riad', 'marrakech', 'roof', 'dar'

In [ ]:
i = 0
for aspect in aspect_keyword:
  i+=1
  if i==8:
    break
  print(aspect, " : ", aspect_keyword[aspect][0:7])

value  :  ['value', 'price', 'tiamo', 'resort', 'fishing', 'basel', 'beach']
room  :  ['room', 'space', 'hotel', 'stay', 'clean', 'staff', 'good']
location  :  ['location', 'locate', 'cabin', 'ankara', 'museum', 'ben', 'haveli']
cleanliness  :  ['clean', 'dirty', 'riad', 'marrakech', 'dar', 'eliane', 'terrace']
service  :  ['service', 'manager', 'butterfly', 'paris', 'sedona', 'basket', 'frank']


In [ ]:
aspect_keyword

defaultdict(None,
            {'cleanliness': ['clean',
              'dirty',
              'riad',
              'marrakech',
              'dar',
              'eliane',
              'terrace',
              'village',
              'picked',
              'bay',
              'whenever',
              'overlooked',
              'istanbul',
              'mosque',
              'blue',
              'courtyard',
              'canyon',
              'mall',
              'beside',
              'cross',
              'tiamo',
              'fishing',
              'port',
              'odor',
              'concrete',
              'cabin',
              'boat',
              'fish',
              'misleading',
              'premise',
              'ski',
              'tall',
              'alongside',
              'activity',
              'resort',
              'da',
              'crossing'],
             'location': ['location',
              'locate',
              'cabi

ABSA

preprocessing

In [ ]:
!pip install pyenchant

     |████████████████████████████████| 55 kB 3.4 MB/s 


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
import nltk
#import enchant
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
import string,pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
import pandas as pd
import json

json.loads(record_list[0])

{'loc': 'Giulianova_Province_of_Teramo_Abruzzo',
 'name': 'Hotel_Baltic',
 'property_dict': {'cleanliness': 3.0,
  'location': 4.0,
  'rooms': 4.0,
  'service': 5.0,
  'sleep quality': 3.0,
  'value': 4.0},
 'text': 'decided family holiday destination saw ranking family hotel review. kept promise. plus staff extremely kind willing help time great attention confort kid private playground front hotel pinery providing shade day long great breakfast beach umbrella deck chair assigned guest beach right front hotel great kid menu also weaning lot activity kid parent well wish free baby club year old swimming pool childrens corner big work fine feel like home two gazebo beach used miniclubs massage available day long spa beach towel provided minus perfection impossible restaurant tends bit slow beach gazebo bigger kid bit small midday sun couple note dont mind first floor avoid elevator peak time there queing booking bit messy better double check make sure everythings fine bed unconfortable t

In [ ]:
review_df_list=[]
for r in range(len(record_list)):
  record=json.loads(record_list[r])
  name=record['name']
  loc=record['loc']
  rating_dict=record['property_dict']
  rating=(rating_dict['cleanliness']+rating_dict['location']+rating_dict['rooms']+rating_dict['service']+rating_dict['value'])/5
  
  sentences=nltk.sent_tokenize(record['text'])
  sentence_list=[]
  for s in sentences:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    sentence_list.append(word_list)

  sentences=[]
  for s in sentence_list:
    if(len(s)!=0):
      sentences.append(' '.join(s))

  txt=""
  for s in sentences:
    txt=txt+" "+s

  review_df_list.append([name,loc,r,txt.split(),rating])
    

review_data=pd.DataFrame(review_df_list,columns=['name','location','id','text','rating'])

In [ ]:
review_data

,name,location,id,text,rating
0,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,"[decided, family, holiday, destination, saw, r...",4.0
1,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,1,"[great, customer, service, good, restaurant, s...",3.6
2,Pousada_Rumo_dos_Ventos,Paraty_State_of_Rio_de_Janeiro,2,"[stayed, rumo, ventos, th, th, november, situa...",4.0
3,Pousada_Rumo_dos_Ventos,Paraty_State_of_Rio_de_Janeiro,3,"[stayed, hotel, april, pleased, beauty, peacef...",5.0
4,Pousada_Rumo_dos_Ventos,Paraty_State_of_Rio_de_Janeiro,4,"[amazing, place, stay, parati, rent, boat, tak...",5.0
...,...,...,...,...,...
49995,Hilton_Omaha,Omaha_Nebraska,49995,"[poor, customer, service, terrible, waitress, ...",2.2
49996,Hilton_Omaha,Omaha_Nebraska,49996,"[supportive, state, largest, industry, agricul...",3.6
49997,Hilton_Omaha,Omaha_Nebraska,49997,"[pleasantly, surprised, modern, aspect, hotel,...",4.8
49998,Hilton_Omaha,Omaha_Nebraska,49998,"[pleased, stay, entire, staff, location, went,...",5.0


In [ ]:

review_sentence_df_list=[]
for r in range(len(record_list)):
  record=json.loads(record_list[r])
  name=record['name']
  loc=record['loc']
  rating_dict=record['property_dict']
  rating=(rating_dict['cleanliness']+rating_dict['location']+rating_dict['rooms']+rating_dict['service']+rating_dict['value'])/5
  
  sentences=nltk.sent_tokenize(record['text'])
  sentence_list=[]
  for s in sentences:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = stopwords.words('english')
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    sentence_list.append(word_list)

  sentences=[]
  for s in sentence_list:
    if(len(s)!=0):
      sentences.append(' '.join(s))

  one_review=[]
  for s in sentences:
    one_review.append([name,loc,r,s,rating])

  review_sentence_df_list+=(one_review)

review_sentence_splited_data=pd.DataFrame(review_sentence_df_list,columns=['name','location','id','text','rating'])

In [ ]:
review_sentence_splited_data

,name,location,id,text,rating
0,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,decided family holiday destination saw ranking...,4.0
1,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,kept promise,4.0
2,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,0,plus staff extremely kind willing help time gr...,4.0
3,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,1,great customer service good restaurant service...,3.6
4,Hotel_Baltic,Giulianova_Province_of_Teramo_Abruzzo,1,giulianuova pretty simple wow town clean beach...,3.6
...,...,...,...,...,...
371902,Hilton_Omaha,Omaha_Nebraska,49998,shuttle bus airport,5.0
371903,Hilton_Omaha,Omaha_Nebraska,49999,hotel great location business,4.0
371904,Hilton_Omaha,Omaha_Nebraska,49999,long line checkin,4.0
371905,Hilton_Omaha,Omaha_Nebraska,49999,room nice staff professional,4.0


In [ ]:
reviews=review_data['text']
reviews.head()

0    [decided, family, holiday, destination, saw, r...
1    [great, customer, service, good, restaurant, s...
2    [stayed, rumo, ventos, th, th, november, situa...
3    [stayed, hotel, april, pleased, beauty, peacef...
4    [amazing, place, stay, parati, rent, boat, tak...
Name: text, dtype: object

----


In [ ]:
from textblob import TextBlob

sentences=[]
aspects=[]
reviewId=[]
sentiments=[]

for i in range(len(sentence_aspect)):
  if len(sentence_aspect[i])!=0:
    try:
      sentences.append(sentences_dup[i])
      sentiments.append(TextBlob(sentences_dup[i]).sentiment.polarity)
      try:
        aspects.append(sentence_aspect[i][0])    
      except Exception as e:
        print(e, " Skipping...")
      
      reviewId.append(review_index[i])
    except:
      pass


In [ ]:
len(sentences_dup)

376020

In [ ]:
len(sentences)

255274

In [ ]:
len(sentence_aspect)

376021

In [ ]:
import pandas as pd

final_df_list=[]
for i in range(len(sentences)):
  temp=[reviewId[i],sentences[i],aspects[i],sentiments[i]]
  final_df_list.append(temp)

final_df=pd.DataFrame(final_df_list,columns=['ReviewId','sentence','aspect','sentiment score'])

In [ ]:
import joblib

joblib.dump(final_df, "drive/My Drive/Winter '22/IR/Project/final_df50k.sav")

["drive/My Drive/Winter '22/IR/Project/final_df50k.sav"]

In [ ]:
final_df.to_csv('final_df.csv')
!cp final_df.csv "drive/My Drive/Winter '22/IR/Project/"

In [ ]:
final_df.head()

,ReviewId,sentence,aspect,sentiment score
0,0,decided family holiday destination saw ranking...,room,0.000000
1,0,plus staff extremely kind willing help time gr...,room,0.211199
2,1,great customer service good restaurant service...,room,0.833333
3,1,giulianuova pretty simple wow town clean beach...,room,0.179167
4,1,baltic simple functional hotel make special pe...,room,0.178571


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split( final_df['sentence'], final_df['aspect'], test_size=0.33, random_state=42)

naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.829619781341627


svm

In [ ]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', svm.SVC(decision_function_shape='ovr')),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
sentences

with cosine similarity

In [ ]:
import gensim
from gensim.models import Word2Vec

aspects_with_cosinesim=[]

for s in sentences:
  for aspect in aspect_keyword.keys():

    max_sim=0
    maxsim_aspect=""
    
    X_set = set(aspect_keyword[aspect]) 
    Y_set = set(s.split())
    l1 =[];
    l2 =[]
     
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: 
          l1.append(1)
        else:
          l1.append(0) 
        if w in Y_set: 
          l2.append(1)
        else:
           l2.append(0)
    c = 0
      
   
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    
    if(cosine>max_sim):
      maxsim_word=aspect
  
  aspects_with_cosinesim.append(maxsim_word)

In [ ]:

final_df_list_withsim=[]
for i in range(len(sentences)):
  temp=[reviewId[i],sentences[i],aspects_with_cosinesim[i],sentiments[i]]
  final_df_list_withsim.append(temp)

final_df_withsim=pd.DataFrame(final_df_list_withsim,columns=['ReviewId','sentence','aspect','sentiment score'])

In [ ]:
final_df_withsim.head()

,ReviewId,sentence,aspect,sentiment score
0,0,decided family holiday destination saw ranking...,room,0.000000
1,0,plus staff extremely kind willing help time gr...,service,0.211199
2,1,great customer service good restaurant service...,service,0.833333
3,1,giulianuova pretty simple wow town clean beach...,cleanliness,0.179167
4,1,baltic simple functional hotel make special pe...,room,0.178571


In [ ]:
joblib.dump(final_df_withsim, "drive/My Drive/Winter '22/IR/Project/final_df_withsim50k.sav")

["drive/My Drive/Winter '22/IR/Project/final_df_withsim50k.sav"]

In [ ]:
#savedf
final_df_withsim.to_csv('final_df_withsim.csv')
!cp final_df_withsim.csv "drive/My Drive/Winter '22/IR/Project/"

In [ ]:
final_df_withsim = pd.read_csv("drive/My Drive/Winter '22/IR/Project/final_df_withsim.csv")

In [ ]:
final_df_withsim.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split( final_df_withsim['sentence'], final_df_withsim['aspect'], test_size=0.33, random_state=42)

naive bayes (with cosine sim)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.7008226398072198
(0.8834332398108664, 0.46241429137809176, 0.5086536643845517, None)


In [ ]:
import joblib

joblib.dump(nb, 'nb_sim.sav')

In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/nb_sim.sav")

["drive/My Drive/Winter '22/IR/Project/nb_sim.sav"]

In [ ]:
nb = joblib.load('nb_sim.sav')
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
!ls

SVM ( with cosine sim )

In [ ]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', svm.SVC(decision_function_shape='ovo')),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
joblib.dump(nb, 'svm_sim.sav')

In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/svm_sim.sav")

In [ ]:
nb = joblib.load('svm_sim.sav')
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
nb.predict(pd.DataFrame([" want clean room"])[0])

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', DecisionTreeClassifier()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9670587955983428
(0.9631295391044631, 0.9550648776262148, 0.9590148267531123, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/DT_sim.sav")

["drive/My Drive/Winter '22/IR/Project/DT_sim.sav"]

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy 0.9597820538692561
(0.964599476505497, 0.9390047377412778, 0.9509511250135075, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/Logistic_sim.sav")

["drive/My Drive/Winter '22/IR/Project/Logistic_sim.sav"]

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', KNeighborsClassifier(n_neighbors=10)),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6844529385928467
(0.7092377189971206, 0.5268476166938528, 0.578912812713755, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/KNN_sim.sav")

["drive/My Drive/Winter '22/IR/Project/KNN.sav"]

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier(n_estimators=10)),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9361949644472407
(0.953647506437213, 0.877730458507618, 0.9115833802556903, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/RForest_sim.sav")

["drive/My Drive/Winter '22/IR/Project/RForest_sim.sav"]